In [1]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

!mkdir -p drive
!google-drive-ocamlfuse drive

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
folder='./drive/ML/Cifar-10/cifar-10-batches-py'

In [0]:
import pandas as pd
import numpy as np
import os
import pickle


In [0]:
def unpickle(file):
  
  '''Load byte data from file'''
  with open(file, 'rb') as f:
    data = pickle.load(f)
    return data


def load_cifar10_data(data_dir):
  '''Return train_data, train_labels, test_data, test_labels
   The shape of data is 32 x 32 x3'''
    
  train_data = None
  train_labels = []

  for i in range(1, 6):
    data_dic = unpickle(data_dir + "/data_batch_{}".format(i))
    if i == 1:
      train_data = data_dic['data']
    else:
      train_data = np.vstack((train_data, data_dic['data']))
    train_labels += data_dic['labels']
  test_data_dic = unpickle(data_dir + "/test_batch")
  test_data = test_data_dic['data']
  test_labels = test_data_dic['labels']

  train_data = train_data.reshape((len(train_data), 3, 32, 32))
  train_data = np.rollaxis(train_data, 1, 4)
  train_labels = np.array(train_labels)

  test_data = test_data.reshape((len(test_data), 3, 32, 32))
  test_data = np.rollaxis(test_data, 1, 4)
  test_labels = np.array(test_labels)

  return train_data, train_labels, test_data, test_labels

In [0]:
train_data, train_labels, test_data, test_labels = load_cifar10_data(folder)


In [6]:
print(train_data.shape)
print(train_labels.shape)

print(test_data.shape)
print(test_labels.shape)


(50000, 32, 32, 3)
(50000,)
(10000, 32, 32, 3)
(10000,)


In [7]:
import keras

Using TensorFlow backend.


In [0]:
y_train=np.asarray(train_labels).reshape(-1,1)
y_train=keras.utils.to_categorical(y_train,num_classes=10)

y_test=np.asarray(test_labels).reshape(-1,1)
y_test=keras.utils.to_categorical(y_test,num_classes=10)



In [0]:
x_train=train_data.astype('float')
x_test=test_data.astype('float')
x_train/=255.0
x_test/=255.0

In [0]:
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout,Input,AveragePooling2D,BatchNormalization,Activation,GlobalAveragePooling2D
from keras.models import Model
from keras.optimizers import SGD,Adam
from keras.losses import binary_crossentropy,categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator

In [11]:
datagen=ImageDataGenerator(featurewise_std_normalization=True,horizontal_flip=True)
datagen.fit(x_train)
train_gen=datagen.flow(x_train,y_train,batch_size=256,shuffle=True)
test_gen=datagen.flow(x_test,y_test,batch_size=256,shuffle=True)

/usr/local/lib/python2.7/dist-packages/keras/preprocessing/image.py:657: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [0]:
def make_model():
    inputs=Input(shape=(32,32,3))
    l=inputs
    l=Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=Conv2D(filters=32,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=MaxPooling2D(pool_size=(2,2))(l)
    
    l=Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=Conv2D(filters=64,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=MaxPooling2D(pool_size=(2,2))(l)
    
    l=Conv2D(filters=96,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=Conv2D(filters=96,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=Conv2D(filters=96,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=MaxPooling2D(pool_size=(2,2))(l)
    
    l=Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=Conv2D(filters=128,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=MaxPooling2D(pool_size=(2,2))(l)
    
    l=Conv2D(filters=196,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=Conv2D(filters=196,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=Conv2D(filters=196,kernel_size=(3,3),strides=(1,1),padding='same')(l)
    l=BatchNormalization()(l)
    l=Activation('relu')(l)
    l=MaxPooling2D(pool_size=(2,2))(l)
    
    l=GlobalAveragePooling2D()(l)
    l=Dropout(0.4)(l)
    l=Dense(256,activation='relu')(l)
    l=Dropout(0.4)(l)
    l=Dense(128,activation='relu')(l)
    l=Dense(10,activation='softmax')(l)
    outputs=l
    return inputs,outputs

In [13]:
train_labels[1]

9

In [21]:
inputs,outputs=make_model()
model=Model(inputs=inputs,outputs=outputs)
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 32, 32, 32)        896       
_________________________________________________________________
batch_normalization_17 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
activation_17 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
batch_normalization_18 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
activation_18 (Activation)   (None, 32, 32, 32)        0         
__________

In [0]:
optim=Adam(1e-3)
loss=categorical_crossentropy
model.compile(optim,loss,metrics=['accuracy'])

In [23]:
model.fit_generator(train_gen,validation_data=test_gen,steps_per_epoch=len(x_train)/256,validation_steps=len(x_test)/256,epochs=20,verbose=1)

Epoch 1/20
195/195 [==============================] - 50s 257ms/step - loss: 1.8486 - acc: 0.2867 - val_loss: 1.7418 - val_acc: 0.4123
Epoch 2/20
195/195 [==============================] - 41s 208ms/step - loss: 1.2661 - acc: 0.5364 - val_loss: 2.0068 - val_acc: 0.4780
Epoch 3/20
 99/195 [==============>...............] - ETA: 18s - loss: 1.0010 - acc: 0.6484

195/195 [==============================] - 40s 204ms/step - loss: 0.9559 - acc: 0.6687 - val_loss: 2.1984 - val_acc: 0.4986
Epoch 4/20
195/195 [==============================] - 40s 207ms/step - loss: 0.7979 - acc: 0.7290 - val_loss: 4.1382 - val_acc: 0.4146
Epoch 5/20
136/195 [===================>..........] - ETA: 11s - loss: 0.7113 - acc: 0.7636

195/195 [==============================] - 40s 205ms/step - loss: 0.7057 - acc: 0.7649 - val_loss: 1.0995 - val_acc: 0.6626
Epoch 6/20
195/195 [==============================] - 40s 205ms/step - loss: 0.6243 - acc: 0.7955 - val_loss: 0.7497 - val_acc: 0.7620
Epoch 7/20
145/195 [=====================>........] - ETA: 9s - loss: 0.5643 - acc: 0.8137

195/195 [==============================] - 41s 210ms/step - loss: 0.5601 - acc: 0.8156 - val_loss: 0.7827 - val_acc: 0.7513
Epoch 8/20
195/195 [==============================] - 41s 212ms/step - loss: 0.5114 - acc: 0.8325 - val_loss: 0.9040 - val_acc: 0.7207
Epoch 9/20
143/195 [=====================>........] - ETA: 10s - loss: 0.4641 - acc: 0.8509

195/195 [==============================] - 42s 213ms/step - loss: 0.4651 - acc: 0.8496 - val_loss: 1.7764 - val_acc: 0.6196
Epoch 10/20
195/195 [==============================] - 41s 211ms/step - loss: 0.4295 - acc: 0.8602 - val_loss: 0.6508 - val_acc: 0.7930
Epoch 11/20
142/195 [====================>.........] - ETA: 10s - loss: 0.3952 - acc: 0.8715

195/195 [==============================] - 42s 213ms/step - loss: 0.3979 - acc: 0.8714 - val_loss: 0.5809 - val_acc: 0.8197
Epoch 12/20
195/195 [==============================] - 42s 215ms/step - loss: 0.3688 - acc: 0.8799 - val_loss: 0.6266 - val_acc: 0.8017
Epoch 13/20
134/195 [===================>..........] - ETA: 12s - loss: 0.3408 - acc: 0.8885

195/195 [==============================] - 43s 223ms/step - loss: 0.3451 - acc: 0.8878 - val_loss: 0.6348 - val_acc: 0.8111
Epoch 14/20
195/195 [==============================] - 44s 225ms/step - loss: 0.3142 - acc: 0.8978 - val_loss: 0.6666 - val_acc: 0.8020
Epoch 15/20
136/195 [===================>..........] - ETA: 12s - loss: 0.3017 - acc: 0.9020

195/195 [==============================] - 44s 224ms/step - loss: 0.3057 - acc: 0.9009 - val_loss: 0.6808 - val_acc: 0.7944
Epoch 16/20
195/195 [==============================] - 42s 218ms/step - loss: 0.2796 - acc: 0.9092 - val_loss: 0.6075 - val_acc: 0.8120
Epoch 17/20
135/195 [===================>..........] - ETA: 11s - loss: 0.2566 - acc: 0.9184

195/195 [==============================] - 41s 209ms/step - loss: 0.2648 - acc: 0.9152 - val_loss: 0.5815 - val_acc: 0.8257
Epoch 18/20
195/195 [==============================] - 41s 211ms/step - loss: 0.2500 - acc: 0.9193 - val_loss: 0.6646 - val_acc: 0.8112
Epoch 19/20
138/195 [====================>.........] - ETA: 11s - loss: 0.2276 - acc: 0.9265

195/195 [==============================] - 41s 208ms/step - loss: 0.2335 - acc: 0.9255 - val_loss: 0.7844 - val_acc: 0.7896
Epoch 20/20
195/195 [==============================] - 41s 208ms/step - loss: 0.2134 - acc: 0.9312 - val_loss: 0.6841 - val_acc: 0.8107


In [0]:
import keras.backend as k

In [0]:
k.set_value(model.optimizer.lr,1e-4)

In [27]:
model.fit_generator(train_gen,validation_data=test_gen,steps_per_epoch=len(x_train)/256,validation_steps=len(x_test)/256,epochs=15,verbose=1)

Epoch 1/15
195/195 [==============================] - 41s 208ms/step - loss: 0.1361 - acc: 0.9587 - val_loss: 0.4491 - val_acc: 0.8688
Epoch 2/15
195/195 [==============================] - 42s 215ms/step - loss: 0.0978 - acc: 0.9697 - val_loss: 0.4646 - val_acc: 0.8739
Epoch 3/15
 99/195 [==============>...............] - ETA: 19s - loss: 0.0820 - acc: 0.9749

195/195 [==============================] - 42s 215ms/step - loss: 0.0795 - acc: 0.9764 - val_loss: 0.5002 - val_acc: 0.8743
Epoch 4/15
195/195 [==============================] - 42s 216ms/step - loss: 0.0737 - acc: 0.9779 - val_loss: 0.5159 - val_acc: 0.8740
Epoch 5/15
118/195 [=================>............] - ETA: 15s - loss: 0.0628 - acc: 0.9819

195/195 [==============================] - 41s 209ms/step - loss: 0.0628 - acc: 0.9818 - val_loss: 0.5215 - val_acc: 0.8747
Epoch 6/15
195/195 [==============================] - 40s 204ms/step - loss: 0.0547 - acc: 0.9835 - val_loss: 0.5597 - val_acc: 0.8743
Epoch 7/15
140/195 [====================>.........] - ETA: 10s - loss: 0.0517 - acc: 0.9844

195/195 [==============================] - 41s 208ms/step - loss: 0.0511 - acc: 0.9845 - val_loss: 0.5856 - val_acc: 0.8768
Epoch 8/15
195/195 [==============================] - 40s 205ms/step - loss: 0.0449 - acc: 0.9867 - val_loss: 0.6189 - val_acc: 0.8711
Epoch 9/15
152/195 [======================>.......] - ETA: 8s - loss: 0.0407 - acc: 0.9880

195/195 [==============================] - 40s 204ms/step - loss: 0.0405 - acc: 0.9879 - val_loss: 0.5985 - val_acc: 0.8779
Epoch 10/15
195/195 [==============================] - 42s 213ms/step - loss: 0.0338 - acc: 0.9902 - val_loss: 0.6524 - val_acc: 0.8713
Epoch 11/15
134/195 [===================>..........] - ETA: 12s - loss: 0.0311 - acc: 0.9913

195/195 [==============================] - 42s 218ms/step - loss: 0.0313 - acc: 0.9912 - val_loss: 0.6706 - val_acc: 0.8762
Epoch 12/15
195/195 [==============================] - 42s 217ms/step - loss: 0.0285 - acc: 0.9915 - val_loss: 0.7005 - val_acc: 0.8699
Epoch 13/15
144/195 [=====================>........] - ETA: 9s - loss: 0.0254 - acc: 0.9925 

195/195 [==============================] - 41s 208ms/step - loss: 0.0260 - acc: 0.9921 - val_loss: 0.7536 - val_acc: 0.8721
Epoch 14/15
195/195 [==============================] - 41s 209ms/step - loss: 0.0245 - acc: 0.9928 - val_loss: 0.7266 - val_acc: 0.8756
Epoch 15/15
139/195 [====================>.........] - ETA: 10s - loss: 0.0222 - acc: 0.9937

195/195 [==============================] - 41s 209ms/step - loss: 0.0232 - acc: 0.9934 - val_loss: 0.7592 - val_acc: 0.8704


In [28]:
model.fit_generator(train_gen,validation_data=test_gen,steps_per_epoch=len(x_train)/256,validation_steps=len(x_test)/256,epochs=10,verbose=1)

Epoch 1/10
195/195 [==============================] - 43s 221ms/step - loss: 0.0171 - acc: 0.9952 - val_loss: 0.7763 - val_acc: 0.8756
Epoch 2/10
195/195 [==============================] - 42s 216ms/step - loss: 0.0192 - acc: 0.9944 - val_loss: 0.7787 - val_acc: 0.8732
Epoch 3/10
 87/195 [============>.................] - ETA: 21s - loss: 0.0176 - acc: 0.9952

195/195 [==============================] - 41s 208ms/step - loss: 0.0172 - acc: 0.9951 - val_loss: 0.8219 - val_acc: 0.8715
Epoch 4/10
195/195 [==============================] - 41s 212ms/step - loss: 0.0207 - acc: 0.9935 - val_loss: 0.7975 - val_acc: 0.8719
Epoch 5/10
125/195 [==================>...........] - ETA: 13s - loss: 0.0164 - acc: 0.9950

195/195 [==============================] - 41s 210ms/step - loss: 0.0162 - acc: 0.9952 - val_loss: 0.8045 - val_acc: 0.8736
Epoch 6/10
195/195 [==============================] - 41s 208ms/step - loss: 0.0138 - acc: 0.9956 - val_loss: 0.8925 - val_acc: 0.8687
Epoch 7/10
137/195 [====================>.........] - ETA: 11s - loss: 0.0155 - acc: 0.9959

195/195 [==============================] - 41s 209ms/step - loss: 0.0154 - acc: 0.9959 - val_loss: 0.9272 - val_acc: 0.8631
Epoch 8/10
195/195 [==============================] - 43s 220ms/step - loss: 0.0157 - acc: 0.9958 - val_loss: 0.8501 - val_acc: 0.8687
Epoch 9/10
128/195 [==================>...........] - ETA: 13s - loss: 0.0140 - acc: 0.9961

195/195 [==============================] - 43s 220ms/step - loss: 0.0132 - acc: 0.9963 - val_loss: 0.8561 - val_acc: 0.8731
Epoch 10/10
195/195 [==============================] - 42s 214ms/step - loss: 0.0138 - acc: 0.9961 - val_loss: 0.8958 - val_acc: 0.8695


In [29]:
resnet34.fit_generator(train_gen,validation_data=test_gen,steps_per_epoch=len(x_train)/256,validation_steps=len(x_test)/256,epochs=15,verbose=1)

Epoch 1/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0248 - acc: 0.9920 - val_loss: 1.0256 - val_acc: 0.8142
Epoch 2/15
195/195 [==============================] - 65s 331ms/step - loss: 0.0217 - acc: 0.9933 - val_loss: 0.8001 - val_acc: 0.8452
Epoch 3/15
104/195 [===============>..............] - ETA: 28s - loss: 0.0180 - acc: 0.9938

195/195 [==============================] - 65s 331ms/step - loss: 0.0194 - acc: 0.9939 - val_loss: 1.0222 - val_acc: 0.8253
Epoch 4/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0177 - acc: 0.9945 - val_loss: 0.8324 - val_acc: 0.8415
Epoch 5/15
143/195 [=====================>........] - ETA: 16s - loss: 0.0168 - acc: 0.9947

195/195 [==============================] - 64s 329ms/step - loss: 0.0165 - acc: 0.9949 - val_loss: 0.8208 - val_acc: 0.8447
Epoch 6/15
195/195 [==============================] - 64s 330ms/step - loss: 0.0156 - acc: 0.9955 - val_loss: 0.9789 - val_acc: 0.8278
Epoch 7/15
152/195 [======================>.......] - ETA: 13s - loss: 0.0173 - acc: 0.9947

195/195 [==============================] - 65s 332ms/step - loss: 0.0173 - acc: 0.9945 - val_loss: 0.8970 - val_acc: 0.8393
Epoch 8/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0125 - acc: 0.9961 - val_loss: 0.8586 - val_acc: 0.8431
Epoch 9/15
154/195 [======================>.......] - ETA: 12s - loss: 0.0098 - acc: 0.9973

195/195 [==============================] - 65s 332ms/step - loss: 0.0094 - acc: 0.9974 - val_loss: 0.8837 - val_acc: 0.8451
Epoch 10/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0126 - acc: 0.9962 - val_loss: 0.9296 - val_acc: 0.8341
Epoch 11/15
153/195 [======================>.......] - ETA: 13s - loss: 0.0105 - acc: 0.9971

195/195 [==============================] - 65s 332ms/step - loss: 0.0101 - acc: 0.9972 - val_loss: 0.8733 - val_acc: 0.8446
Epoch 12/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0079 - acc: 0.9978 - val_loss: 0.9334 - val_acc: 0.8389
Epoch 13/15
153/195 [======================>.......] - ETA: 12s - loss: 0.0065 - acc: 0.9983

195/195 [==============================] - 65s 332ms/step - loss: 0.0065 - acc: 0.9983 - val_loss: 0.9006 - val_acc: 0.8446
Epoch 14/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0084 - acc: 0.9978 - val_loss: 0.8981 - val_acc: 0.8477
Epoch 15/15
153/195 [======================>.......] - ETA: 12s - loss: 0.0045 - acc: 0.9991

195/195 [==============================] - 65s 332ms/step - loss: 0.0050 - acc: 0.9989 - val_loss: 0.9123 - val_acc: 0.8458


In [31]:
resnet34.fit_generator(train_gen,validation_data=test_gen,steps_per_epoch=len(x_train)/256,validation_steps=len(x_test)/256,epochs=15,verbose=1)

Epoch 1/15
195/195 [==============================] - 65s 333ms/step - loss: 0.0034 - acc: 0.9993 - val_loss: 0.8947 - val_acc: 0.8536
Epoch 2/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0031 - acc: 0.9994 - val_loss: 0.8973 - val_acc: 0.8494
Epoch 3/15
104/195 [===============>..............] - ETA: 28s - loss: 0.0023 - acc: 0.9996

195/195 [==============================] - 65s 333ms/step - loss: 0.0021 - acc: 0.9997 - val_loss: 0.8787 - val_acc: 0.8562
Epoch 4/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0018 - acc: 0.9997 - val_loss: 0.8945 - val_acc: 0.8534
Epoch 5/15
143/195 [=====================>........] - ETA: 15s - loss: 0.0019 - acc: 0.9996

195/195 [==============================] - 64s 329ms/step - loss: 0.0018 - acc: 0.9996 - val_loss: 0.9130 - val_acc: 0.8540
Epoch 6/15
195/195 [==============================] - 64s 329ms/step - loss: 9.7837e-04 - acc: 0.9999 - val_loss: 0.8760 - val_acc: 0.8565
Epoch 7/15
147/195 [=====================>........] - ETA: 14s - loss: 9.9030e-04 - acc: 0.9999

195/195 [==============================] - 65s 332ms/step - loss: 9.3300e-04 - acc: 0.9999 - val_loss: 0.8935 - val_acc: 0.8558
Epoch 8/15
195/195 [==============================] - 65s 332ms/step - loss: 8.3467e-04 - acc: 1.0000 - val_loss: 0.8982 - val_acc: 0.8557
Epoch 9/15
143/195 [=====================>........] - ETA: 15s - loss: 6.1183e-04 - acc: 1.0000

195/195 [==============================] - 64s 326ms/step - loss: 5.9154e-04 - acc: 1.0000 - val_loss: 0.8911 - val_acc: 0.8582
Epoch 10/15
195/195 [==============================] - 64s 327ms/step - loss: 5.3548e-04 - acc: 1.0000 - val_loss: 0.8803 - val_acc: 0.8561
Epoch 11/15
141/195 [====================>.........] - ETA: 16s - loss: 5.4815e-04 - acc: 1.0000

195/195 [==============================] - 64s 331ms/step - loss: 5.2250e-04 - acc: 1.0000 - val_loss: 0.9188 - val_acc: 0.8590
Epoch 12/15
195/195 [==============================] - 64s 329ms/step - loss: 4.8260e-04 - acc: 1.0000 - val_loss: 0.8889 - val_acc: 0.8568
Epoch 13/15
140/195 [====================>.........] - ETA: 17s - loss: 4.9449e-04 - acc: 1.0000

195/195 [==============================] - 65s 332ms/step - loss: 4.9542e-04 - acc: 1.0000 - val_loss: 0.8876 - val_acc: 0.8575
Epoch 14/15
195/195 [==============================] - 65s 332ms/step - loss: 0.0012 - acc: 0.9998 - val_loss: 0.9170 - val_acc: 0.8526
Epoch 15/15
144/195 [=====================>........] - ETA: 15s - loss: 5.9299e-04 - acc: 1.0000

195/195 [==============================] - 65s 332ms/step - loss: 6.2380e-04 - acc: 1.0000 - val_loss: 0.9072 - val_acc: 0.8572


In [0]:
resnet34.fit_generator(train_gen,validation_data=test_gen,steps_per_epoch=len(x_train)/256,validation_steps=len(x_test)/256,epochs=20,verbose=1)

Epoch 1/20
195/195 [==============================] - 65s 333ms/step - loss: 4.1787e-04 - acc: 1.0000 - val_loss: 0.9019 - val_acc: 0.8575
Epoch 2/20
195/195 [==============================] - 65s 333ms/step - loss: 4.0950e-04 - acc: 1.0000 - val_loss: 0.8976 - val_acc: 0.8626
Epoch 3/20
 94/195 [=============>................] - ETA: 31s - loss: 3.6737e-04 - acc: 1.0000

195/195 [==============================] - 65s 332ms/step - loss: 3.7145e-04 - acc: 1.0000 - val_loss: 0.9229 - val_acc: 0.8593
Epoch 4/20
195/195 [==============================] - 65s 332ms/step - loss: 6.1552e-04 - acc: 0.9999 - val_loss: 0.9344 - val_acc: 0.8575
Epoch 5/20
131/195 [===================>..........] - ETA: 19s - loss: 4.4524e-04 - acc: 1.0000

195/195 [==============================] - 65s 333ms/step - loss: 4.5950e-04 - acc: 1.0000 - val_loss: 0.9228 - val_acc: 0.8564
Epoch 6/20
195/195 [==============================] - 65s 332ms/step - loss: 4.8111e-04 - acc: 1.0000 - val_loss: 0.9260 - val_acc: 0.8538
Epoch 7/20
140/195 [====================>.........] - ETA: 17s - loss: 4.7697e-04 - acc: 1.0000

195/195 [==============================] - 65s 332ms/step - loss: 4.2727e-04 - acc: 1.0000 - val_loss: 0.9469 - val_acc: 0.8551
Epoch 8/20
195/195 [==============================] - 65s 332ms/step - loss: 3.6268e-04 - acc: 1.0000 - val_loss: 0.9178 - val_acc: 0.8545
Epoch 9/20
141/195 [====================>.........] - ETA: 16s - loss: 3.1579e-04 - acc: 1.0000

195/195 [==============================] - 64s 328ms/step - loss: 3.1671e-04 - acc: 1.0000 - val_loss: 0.9048 - val_acc: 0.8591
Epoch 10/20
195/195 [==============================] - 64s 327ms/step - loss: 2.7627e-04 - acc: 1.0000 - val_loss: 0.9461 - val_acc: 0.8533
Epoch 11/20
141/195 [====================>.........] - ETA: 16s - loss: 6.1395e-04 - acc: 0.9999

195/195 [==============================] - 64s 327ms/step - loss: 5.8010e-04 - acc: 0.9999 - val_loss: 0.9521 - val_acc: 0.8574
Epoch 12/20
195/195 [==============================] - 64s 330ms/step - loss: 3.2594e-04 - acc: 1.0000 - val_loss: 0.9423 - val_acc: 0.8585
Epoch 13/20
140/195 [====================>.........] - ETA: 17s - loss: 2.9423e-04 - acc: 1.0000

195/195 [==============================] - 65s 333ms/step - loss: 2.9134e-04 - acc: 1.0000 - val_loss: 0.9457 - val_acc: 0.8574
Epoch 14/20
195/195 [==============================] - 65s 331ms/step - loss: 2.4101e-04 - acc: 1.0000 - val_loss: 0.9392 - val_acc: 0.8567
Epoch 15/20
 68/195 [=========>....................] - ETA: 39s - loss: 2.3387e-04 - acc: 1.0000

In [34]:
resnet34.fit_generator(train_gen,validation_data=test_gen,steps_per_epoch=len(x_train)/256,validation_steps=len(x_test)/256,epochs=20,verbose=1)

Epoch 1/20
195/195 [==============================] - 64s 330ms/step - loss: 1.6122e-04 - acc: 1.0000 - val_loss: 0.9262 - val_acc: 0.8583
Epoch 2/20
195/195 [==============================] - 64s 330ms/step - loss: 1.8567e-04 - acc: 1.0000 - val_loss: 0.9501 - val_acc: 0.8579
Epoch 3/20
 94/195 [=============>................] - ETA: 30s - loss: 2.4403e-04 - acc: 1.0000

195/195 [==============================] - 64s 326ms/step - loss: 2.2040e-04 - acc: 1.0000 - val_loss: 0.9374 - val_acc: 0.8567
Epoch 4/20
195/195 [==============================] - 64s 330ms/step - loss: 1.8332e-04 - acc: 1.0000 - val_loss: 0.9325 - val_acc: 0.8586
Epoch 5/20
131/195 [===================>..........] - ETA: 19s - loss: 1.7314e-04 - acc: 1.0000

195/195 [==============================] - 64s 329ms/step - loss: 1.7265e-04 - acc: 1.0000 - val_loss: 0.9439 - val_acc: 0.8567
Epoch 6/20
195/195 [==============================] - 64s 329ms/step - loss: 1.5886e-04 - acc: 1.0000 - val_loss: 0.9559 - val_acc: 0.8569
Epoch 7/20
139/195 [====================>.........] - ETA: 17s - loss: 2.4571e-04 - acc: 1.0000

195/195 [==============================] - 64s 330ms/step - loss: 2.2773e-04 - acc: 1.0000 - val_loss: 0.9375 - val_acc: 0.8590
Epoch 8/20
195/195 [==============================] - 64s 327ms/step - loss: 9.2803e-04 - acc: 0.9999 - val_loss: 0.9617 - val_acc: 0.8545
Epoch 9/20
143/195 [=====================>........] - ETA: 15s - loss: 3.5427e-04 - acc: 1.0000

195/195 [==============================] - 64s 329ms/step - loss: 3.2067e-04 - acc: 1.0000 - val_loss: 0.9478 - val_acc: 0.8571
Epoch 10/20
195/195 [==============================] - 64s 327ms/step - loss: 4.0010e-04 - acc: 0.9999 - val_loss: 1.0621 - val_acc: 0.8444
Epoch 11/20
141/195 [====================>.........] - ETA: 16s - loss: 3.2264e-04 - acc: 1.0000

195/195 [==============================] - 64s 327ms/step - loss: 3.1245e-04 - acc: 1.0000 - val_loss: 0.9734 - val_acc: 0.8576
Epoch 12/20
195/195 [==============================] - 65s 332ms/step - loss: 8.1266e-04 - acc: 0.9998 - val_loss: 0.9816 - val_acc: 0.8523
Epoch 13/20
141/195 [====================>.........] - ETA: 16s - loss: 4.0561e-04 - acc: 1.0000

195/195 [==============================] - 64s 329ms/step - loss: 3.9414e-04 - acc: 1.0000 - val_loss: 0.9575 - val_acc: 0.8560
Epoch 14/20
195/195 [==============================] - 65s 332ms/step - loss: 2.5020e-04 - acc: 1.0000 - val_loss: 0.9718 - val_acc: 0.8552
Epoch 15/20
140/195 [====================>.........] - ETA: 17s - loss: 2.2314e-04 - acc: 1.0000

195/195 [==============================] - 65s 333ms/step - loss: 2.1825e-04 - acc: 1.0000 - val_loss: 0.9700 - val_acc: 0.8570
Epoch 16/20
195/195 [==============================] - 64s 330ms/step - loss: 2.1104e-04 - acc: 1.0000 - val_loss: 0.9467 - val_acc: 0.8575
Epoch 17/20
140/195 [====================>.........] - ETA: 16s - loss: 1.8078e-04 - acc: 1.0000

195/195 [==============================] - 65s 331ms/step - loss: 1.9524e-04 - acc: 1.0000 - val_loss: 0.9487 - val_acc: 0.8575
Epoch 18/20
195/195 [==============================] - 65s 331ms/step - loss: 1.9473e-04 - acc: 1.0000 - val_loss: 0.9380 - val_acc: 0.8583
Epoch 19/20
140/195 [====================>.........] - ETA: 16s - loss: 1.6294e-04 - acc: 1.0000

195/195 [==============================] - 65s 331ms/step - loss: 1.7188e-04 - acc: 1.0000 - val_loss: 0.9611 - val_acc: 0.8578
Epoch 20/20
195/195 [==============================] - 65s 332ms/step - loss: 1.7350e-04 - acc: 1.0000 - val_loss: 0.9833 - val_acc: 0.8562
